In [ ]:
# 주피터 노트북 환경설정
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

from IPython.core.display import display, HTML
# display(HTML("<style>.container { font-weight: bold !important; font-family:'Malgun Gothic' !important;}</style>"))
display(HTML("<style>.container { font-weight: bold !important;}</style>"))
display(HTML("<style>.container { width: 98% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.linear_model import Ridge, Lasso

from sklearn.metrics import f1_score, recall_score, precision_score, classification_report

from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.pipeline import Pipeline

## 릿지와 라쏘

#### 규제(Rugularization) 
- 머신러닝 모델이 훈련 세트를 너무 과도하게 학습하지 못하도록 제어하는 것. 선형 회귀의 경우 계수 즉 기눌의 크기를 작게 만든다.  

#### 릿지(ridge) 회귀 
- 규제가 있는 선형 회귀 모델중 하나이며 선형 모델의 계수를 작게 만들어 과대 적합을 완화시킨다.계수의 제곱 기준 

#### 라쏘(lasso) 회귀 
- 릿지와 같이 규제가 있는 선형 회귀 모델중 하나이며 회귀 계수 값을 0으로 만들수도 있다. 계수의 절대값 기준

## 릿지

In [ ]:
df = pd.read_csv('https://bit.ly/perch_csv')
perch_full = df.to_numpy()

perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 
     1000.0, 1000.0]
     )

X_train, X_test, y_train, y_test = train_test_split(perch_full, perch_weight, random_state=42)

poly = PolynomialFeatures(degree=5, include_bias=False)

poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)


scale = StandardScaler()
scale.fit(X_train_poly)

X_train_scaled = scale.transform(X_train_poly)
X_test_scaled = scale.transform(X_test_poly)

In [ ]:
# from sklearn.linear_model import Ridge

model_ridge = Ridge()
model_ridge.fit(X_train_scaled, y_train)
print(model_ridge.score(X_train_scaled, y_train))

In [ ]:
print(model_ridge.score(X_test_scaled, y_test))

In [ ]:
model_ridge.coef_

In [ ]:
model_ridge.coef_.min(), model_ridge.coef_.max(), model_ridge.coef_.mean()

In [ ]:
train_score = []
test_score = []

In [ ]:
alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
for alpha in alpha_list:
    
    model_ridge = Ridge(alpha=alpha)
    
    model_ridge.fit(X_train_scaled, y_train)
    
    train_score.append(model_ridge.score(X_train_scaled, y_train))
    test_score.append(model_ridge.score(X_test_scaled, y_test))

In [ ]:
alpha_list

In [ ]:
plt.plot(alpha_list, train_score)
plt.plot(alpha_list, test_score)
plt.show()

In [ ]:
alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
print(np.log10(alpha_list))
print(10**(np.log10(alpha_list)))

In [ ]:
np.set_printoptions(precision=3, suppress=True)
10**(np.log10(alpha_list))

In [ ]:
plt.plot(np.log10(alpha_list), train_score, label='Train')
plt.plot(np.log10(alpha_list), test_score, label='Test')
plt.show()

In [ ]:
model_ridge = Ridge(alpha=0.1)
model_ridge.fit(X_train_scaled, y_train)

print(model_ridge.score(X_train_scaled, y_train))
print(model_ridge.score(X_test_scaled, y_test))

In [ ]:
model_ridge.coef_

In [ ]:
model_ridge.coef_.min(), model_ridge.coef_.max(), model_ridge.coef_.mean()

## 라쏘

In [ ]:
# from sklearn.linear_model import Lasso

model_lasso = Lasso()
model_lasso.fit(X_train_scaled, y_train)
print(model_lasso.score(X_train_scaled, y_train))

In [ ]:
model_lasso.coef_

In [ ]:
print(model_lasso.score(X_test_scaled, y_test))

In [ ]:
train_score = []
test_score = []

alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
for alpha in alpha_list:
    # 라쏘 모델을 만듭니다
    model_lasso = Lasso(alpha=alpha, max_iter=10000)
    # 라쏘 모델을 훈련합니다
    model_lasso.fit(X_train_scaled, y_train)
    # 훈련 점수와 테스트 점수를 저장합니다
    train_score.append(model_lasso.score(X_train_scaled, y_train))
    test_score.append(model_lasso.score(X_test_scaled, y_test))

In [ ]:
# np.log10() 이용 
alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
# 1일때 가장 점수 차이가 적게난다. = 10**(1)

plt.plot(np.log10(alpha_list), train_score, marker='o')
plt.plot(np.log10(alpha_list), test_score, marker='o')
plt.show()

In [ ]:
model_lasso2 = Lasso(alpha=10)
model_lasso2.fit(X_train_scaled, y_train)

print(model_lasso2.score(X_train_scaled, y_train))
print(model_lasso2.score(X_test_scaled, y_test))

In [ ]:
model_lasso2.coef_

In [ ]:
print(np.sum(model_lasso2.coef_ == 0))

# 퀴즈 

- 성별 키와 몸무게(weight-height.csv) 데이타셋을 이용하여 선형, 다항적용, 릿지, 라쏘 모델별로 테스트하고 결과를 확인하여라. 
